<a href="https://colab.research.google.com/github/hoangnhan12-arc/CS114.K21.KHTN/blob/master/SacasmDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#            SARCASM DETECTION
NHÓM:
1. THÁI HOÀNG NHÂN - 1852
2. NGUYỄN VƯƠNG THỊNH - 18520367







# 1. MÔ TẢ BÀI TOÁN VÀ CÁCH THU THẬP DỮ LIỆU:
- MÔ TẢ BÀI TOÁN

  Do **Sarcasm Detection**(phát hiện châm biếm) từng thu thập dataset từ **Twitter** trên **hashtag**. Do có tweets liên quan với nhau và có nhiều bộ data gây nhiễu. Để khắc phục vấn đề, người ta dùng dataset được thu thập từ 2 trang web **TheOnion** và **HuffPost**. Data từ **TheOnion** thì hoàn toàn là châm biếm còn Data từ **HuffPost** thì là chính thống, không châm biếm. Các data được thu thập là các **headlines**(tiêu đề) của các bài báo

  **Ưu điểm:**
 - Vì là các bài báo nên ngôn từ sẽ chuẩn mực, ít bị sai chính tả, ngữ pháp
 - Do các headlines trong các bài báo sẽ độc lập với nhau không như các tweets liên quan với nhau
 - Do 2 web thường xuyên ra những bài báo mới nên sẽ có bộ data chất lượng hơn, ít gây nhiễu hơn

- CÁCH THU THẬP DỮ LIỆU:
 - Crawl data từ 2 trang web **HuffPost** và **TheOnion** chứa ở 2 file json riêng
 - Gộp 2 file json lại và tiến hành "xáo trộn" lại dữ liệu

#2. THU THẬP HƠN 2000 HEADLINES

# CRAWL DATA

- Công cụ: Scrapy
- Nguồn crawl:
  - https://www.theonion.com/ 
  - https://www.huffpost.com/

# SETTING NOTEBOOK- INSTALLING SCRAPY

In [ ]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
!pip install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess

# CRAWL "https://www.theonion.com/"

1. Class ghi data vào file json

In [ ]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('theonion.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


2. Class crawl data từ "https://www.theonion.com/"
- Do web có phần chuyển sang trang kế tiếp nên chúng em crawl trong phần **Elements** trong **Inspect** và theo đường link dẫn tới trang kế tiếp để crawl
- Các mục chúng em crawl trong web là:
 - policitcs  "https://politics.theonion.com/"
 - entertainment "https://entertainment.theonion.com/"

In [ ]:
import logging
class TheOnionSpider(scrapy.Spider):
    name = "TheOnion"
    start_urls = [
      "https://politics.theonion.com/",
      "https://entertainment.theonion.com/"
    ]
    # Tạo luồng pipelines
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',                                
        'FEED_URI': 'theonion.json'                        
    }
    
    def parse(self, response):
        flag = 1
        for article in response.css('article.cw4lnv-0.iTueKC.js_post_item'):
            # Lọc các article có năm phát hành từ 2016 đến nay
            if article.css('div.sc-3nbvzd-1.kpXIm::text').extract_first()[-2:] < '16' :
              flag = 0
              break
              # Lọc các article có sẵn link
            elif article.css('figure.sc-1xh12qx-0.vvKzB.cw4lnv-1.fKLmwI a::attr(href)').extract_first() is not None:
              yield {
                 #Trích dẫn các đường link và tiêu đề 
                'article_link': article.css('figure.sc-1xh12qx-0.vvKzB.cw4lnv-1.fKLmwI a::attr(href)').extract_first(),
                'headline': article.css('h2.sc-759qgu-0.cYlVdn.cw4lnv-6.eXwNRE::text').extract_first(),
                'is_sarcastic': 1
            }
        # Crawl theo đường dẫn qua trang khác
        if flag == 1:
          next_page = response.css('div.sc-1uzyw0z-0.kNHeFZ a::attr(href)').get()
          if next_page is not None:
              next_page = response.urljoin(next_page)
              yield scrapy.Request(next_page, callback=self.parse)


3. Chạy Crawler

In [ ]:
process = CrawlerProcess()
process.crawl(TheOnionSpider)
process.start()


2020-06-25 14:34:13 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: scrapybot)
2020-06-25 14:34:13 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
2020-06-25 14:34:13 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-06-25 14:34:13 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}
/usr/local/lib/python3.6/dist-packages/scrapy/extensions/feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)


4. Xuất file json vừa ghi

In [ ]:
import pandas as pd
dfjson = pd.read_json('theonion.json')
print(dfjson)

                                           article_link  ... is_sarcastic
0     https://entertainment.theonion.com/tuba-player...  ...            1
1     https://entertainment.theonion.com/your-horosc...  ...            1
2     https://entertainment.theonion.com/your-horosc...  ...            1
3     https://entertainment.theonion.com/what-to-rea...  ...            1
4     https://entertainment.theonion.com/your-horosc...  ...            1
...                                                 ...  ...          ...
1145  https://politics.theonion.com/revelations-from...  ...            1
1146  https://politics.theonion.com/whos-speaking-at...  ...            1
1147  https://politics.theonion.com/whos-speaking-at...  ...            1
1148  https://politics.theonion.com/whos-speaking-at...  ...            1
1149  https://politics.theonion.com/whos-speaking-at...  ...            1

[1150 rows x 3 columns]


# CRAWL "https://www.huffpost.com/"
1. Class ghi data vào file json

In [ ]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('huffpost.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


2. Class crawl data từ "https://www.huffpost.com/"
- Do web chỉ có nút "load more" nên chúng em crawl trong phần **Network** trong **Inspect**. 
- Khi bấm vào nút "load more" sẽ có đường dẫn 'https://www.huffpost.com/api/department/news/cards?page=2&limit=10' hiện trong **Network**. Tụi em sẽ lấy đường dẫn này để crawl các trang tiếp theo bằng cách gán số vào "page=" trong đường dẫn  
- Các mục chúng em crawl trong web là:
 - policitcs  "https://www.huffpost.com/api/section/politics/cards?page=2&limit=10"
 - entertainment "https://www.huffpost.com/api/department/entertainment/cards?page=2&limit=10"
 - work/life và wellness trong life là "https://www.huffpost.com/api/section/worklife/cards?page=2&limit=10" và "https://www.huffpost.com/api/section/healthy-living/cards?page=2&limit=10" 


In [ ]:
import logging
class HuffPostSpider(scrapy.Spider):
    name = "HuffPost"
    base_url = 'https://www.huffpost.com/api/department/news/cards?page=%s&limit=10'
    start_urls = [base_url % 1]
    
    #Tạo luồng pipelines    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'huffpost.json'                        
    }
    def parse(self, response):

        Links = ['https://www.huffpost.com/api/section/politics/cards?page=%s&limit=10', 
                'https://www.huffpost.com/api/department/entertainment/cards?page=%s&limit=10',
                'https://www.huffpost.com/api/section/worklife/cards?page=%s&limit=10',
                'https://www.huffpost.com/api/section/healthy-living/cards?page=%s&limit=10']
        # Crawl theo đường dẫn
        for link in Links:
             for page in range(1,200):
                yield scrapy.Request(link % page, callback = self.parseNextPage)

    def parseNextPage(self, response):

      data = json.loads(response.body)
      for article in data['cards']:
            
            yield {
                #Trích dẫn các đường link và tiêu đề
               'article_link': article['headlines'][0]['url'] ,
               'headline': article['headlines'][0]['text'], 
               'is_sarcastic': 0 
            }


3. Chạy crawler

In [ ]:
process = CrawlerProcess({'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'})
process.crawl(HuffPostSpider)
process.start()

2020-06-25 14:58:21 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: scrapybot)
2020-06-25 14:58:21 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
2020-06-25 14:58:21 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-06-25 14:58:21 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}
/usr/local/lib/python3.6/dist-packages/scrapy/extensions/feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more detail

4. Xuất file json vừa ghi

In [ ]:
import pandas as pd
dfjson = pd.read_json('huffpost.json')
print(dfjson)

                                           article_link  ... is_sarcastic
0     https://www.huffpost.com/entry/aclu-john-bolto...  ...            0
1     https://www.huffpost.com/entry/lara-trump-dona...  ...            0
2     https://www.huffpost.com/entry/don-winslow-don...  ...            0
3     https://www.huffpost.com/entry/us-needs-repara...  ...            0
4     https://www.huffpost.com/entry/william-cohen-d...  ...            0
...                                                 ...  ...          ...
1353  https://www.huffpost.com/entry/andrew-yang-don...  ...            0
1354  https://www.huffpost.com/entry/fox-news-tucker...  ...            0
1355  https://www.huffpost.com/entry/coronavirus-cas...  ...            0
1356  https://www.huffpost.com/entry/oakland-schools...  ...            0
1357  https://www.huffpost.com/entry/william-barr-ju...  ...            0

[1358 rows x 3 columns]


# MERGE FILES.JSON AND SHUFFLE FILE.JSON

In [ ]:

import random 
  
with open('theonion.json','r') as json_file:
    theonion = json.load(json_file)

with open('huffpost.json','r') as json_file:
    huffpost = json.load(json_file)

Sarcasm_Headlines_Dataset= theonion + huffpost
random.shuffle(Sarcasm_Headlines_Dataset)

with open('Sarcasm_Headlines_Dataset_v2.json', 'w') as json_file:
    json.dump(Sarcasm_Headlines_Dataset, json_file)
 

# TRAINNING MODEL

In [ ]:
from google.colab import files
file = files.upload()

Saving Sarcasm_Headlines_Dataset.json to Sarcasm_Headlines_Dataset.json


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd 

data = pd.read_json("Sarcasm_Headlines_Dataset.json",lines=True)

X_train = data['headline'][0:20000]
Y_train = data['is_sarcastic'][0:20000]
XX_test = data['headline'][20001:]
YY_test = data['is_sarcastic'][20001:]

X_train = X_train.values
XX_test = XX_test.values

In [20]:
print(X_train.shape)
print(XX_test.shape)

(20000,)
(6708,)


In [21]:
#creat train countvecotrize 
cv_train = CountVectorizer()

X_train = cv_train.fit_transform(X_train)
#convert to array
X_train = X_train.toarray()
 
#creat test countvecotrize 
cv_test = CountVectorizer(vocabulary=cv_train.get_feature_names())

#convert to array
XX_test = cv_test.fit_transform(XX_test).toarray()

In [22]:
test = pd.read_json("Sarcasm_Headlines_Dataset_v2.json")

In [23]:
print(type(test['headline']))

<class 'pandas.core.series.Series'>


In [24]:
X_test = test['headline']
X_test=X_test.values
Y_test = test['is_sarcastic']

In [25]:
#convert to array
X_test = cv_test.fit_transform(X_test).toarray()

In [31]:
model = LogisticRegression()

#training
model.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
#predict
predictions1_1 = model.predict(XX_test)
predictions2 = model.predict(X_test)

In [33]:
#traing
model = LinearSVC()
model.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [34]:
#predict
predictions1_2 = model.predict(XX_test)
predictions3 = model.predict(X_test)

In [36]:
print(classification_report(YY_test,predictions1_1))
#using LinearRegression with old test_data
print(classification_report(Y_test,predictions2))
#using LinearRegression with new test_data
print(classification_report(YY_test,predictions1_2))
#using LinearSVC with old test_data
print(classification_report(Y_test,predictions3))
#using LinearSVC with new test_data

              precision    recall  f1-score   support

           0       0.84      0.87      0.86      3779
           1       0.83      0.79      0.81      2929

    accuracy                           0.84      6708
   macro avg       0.84      0.83      0.83      6708
weighted avg       0.84      0.84      0.84      6708

              precision    recall  f1-score   support

           0       0.68      0.88      0.77      1358
           1       0.79      0.51      0.62      1150

    accuracy                           0.71      2508
   macro avg       0.73      0.70      0.69      2508
weighted avg       0.73      0.71      0.70      2508

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      3779
           1       0.81      0.76      0.79      2929

    accuracy                           0.82      6708
   macro avg       0.82      0.81      0.81      6708
weighted avg       0.82      0.82      0.82      6708

              preci

PREDICT ONE INPUT STRING


In [37]:
import numpy as np
n = input()
n = [n]

n = np.asarray(n)


n = cv_test.fit_transform(n).toarray()
predict = model.predict(n)
if predict==1:
  print("=> Is sarcastic")
else:
  print("=> Isn't sarcastic")


Donald Trump is stupid
=> Isn't sarcastic


1+2)

Đã mô tả trong file ipynb

3) Xử lí dữ liệu, feature engineering trên dataset đã cho:

- Nhìn chung, dataset đã cho khá ổn ( đều là chữ viết thường, không chưa các dấu như “,” , “.”, …) nên ta không cần xử lí cũng được
- Feature engineering dataset đã cho:
+ Sử dụng CountVectorize của sklearn để chuyển chuổi sang vector với headlines tập train và test.
+ Với tập train, ta chỉ cần sử dụng method CountVectorize().fit_transform().toarray() để convert
+Với tập test,  vì đây là tập dữ liệu test, nó phải đồng bộ hóa với vocabulary của tập train. Nói một cách đơn giản là nếu như ta không đồng bộ hóa tập test với vocabulary tập train thì nó sẽ có thể không cùng 1 kích thước để có thể predict được. Do đó ta phải xây dựng method CountVectorize() cho riêng tập test dựa trên vocabulary của tập train bằng cách tạo 1 variable countvectorize_test mới với cú pháp như sau: countvectorize_test = CountVectorize(vocabulary=countvectorize_train.get_feature_name()), rồi sau đó convert tập test sang array như bình thường bằng lệnh: countvectorize_test().fit_transform().toarray()

4) Mô tả quá trình chọn model, học

- Nhóm đã thử qua một vài classification model như LogisticRegression, KNeighborsClassifier, DecisionTreeClassifier, LinearSVC thì KNeighborsClassifier và DecisionTreeClassifier thì bị tràn RAM trên google colab, còn LogisticRegression và LinearSVC thì chạy được và đạt kết quả gần như nhau nên nhóm sẽ demo cả 2 model
- Qúa trình traing sử dụng method cho sẵn của thư viện sklearn là :
Model = LogisticRegression()
Model.fit(X_train, Y_train)

5) Mô tả cách dùng model đã train để viết một đoạn chương trình ngắn, thực hiện sacarsm detection cho một headline bất kỳ được nhập vào

Được thực hiện trong phần code

6)  Đánh giá:
- Ta dễ dàng nhận thấy rằng cho dù là dùng LinearRegression hay LinearSVC thì trên test_data cũ đều ra kết quả tốt hơn so với test_data mới. Bởi vì test_data cũ nó có những nội dung gần với train_data hơn test_data mới
- LinearRegression hay LinearSVC đều cho ra 2 kết quả gần như nhau
=> Bài toán này phụ thuộc rất nhiều vào train_data, nếu train_data trải đều trên các năm thì ta sẽ được một kết quả tốt hơn. Một ví dụ rõ ràng trong test của nhóm đó là: Donald Trump is st*pi* và Donald Trump is not st*pi* (để dấu * vì không dám ghi ra ) thì kết quả đều in ra là isn’t sarcastic. Vì đơn giản, train_data không chứa vocabulary “Donald Trump” 



